In [ ]:
# from libs import stable_diffusion_service
from libs.stable_diffusion_service import GenerateImage, StableDiffusion
from PIL import Image
from IPython.display import display
from tqdm import tqdm
from tqdm.asyncio import tqdm as async_tqdm
import asyncio
import time
import os

# STABLE_DIFFUSION_BASE_URL = "http://100.119.72.3:7860" # 腾讯云IDE
STABLE_DIFFUSION_BASE_URL = "http://192.168.10.101:7860" # 笔记本
# STABLE_DIFFUSION_BASE_URL = "http://localhost:6006" # AutoDL

SD = StableDiffusion(STABLE_DIFFUSION_BASE_URL,concurrency_limit=10)
is_SD_preheated = True

In [ ]:
character_list: list[str] = [
    r"amiya \(arknights\)",
    r"texas \(arknights\)",
    r"skadi \(arknights\)",
    # r"suzuran \(arknights\)",
    # r"kaltsit \(arknights\)",
    # r"exusiai \(arknights\)",
    # r"w \(arknights\)",
    # r"dusk \(arknights\)",
    # r"goldenglow \(arknights\)",
    # r"irene \(arknights\)",
    # r"sussurro \(arknights\)",
    # r"hoshiguma \(arknights\)",
    # r"blue poison \(arknights\)",
    # r"eyjafjalla \(arknights\)",
    # r"schwarz \(arknights\)",
    # r"siege \(arknights\)",
]
prompt_list: list[dict] = []
for character in character_list:
    prompt_list.append(
        {
            "prompt": r"1girl,"
            + character
            + r",arknights,safe,solo,chyoel,solar \(happymonk\),[kuzuvine],[[dino \(dinoartforame\)]],[[[ciloranko]]],ask \(askzy\),masterpiece,high score,great score,absurdres,year 2025,",
            "negative_prompt": r"text,watermark,bad anatomy,bad proportions,extra limbs,extra digit,extra legs,extra legs and arms,disfigured,missing arms,too many fingers,fused fingers,missing fingers,unclear eyes,watermark,username,logo,artist logo,patreon logo,weibo logo,arknights logo,",
            "width": 832,
            "height": 1216,
            "cfg_scale": 5,
            "steps": 28,
            "sampler_name": "Euler a",
            "seed": -1,
            "batch_size": 2,
        }
    )

In [ ]:
# 预热模型
if not is_SD_preheated:
    SD.generate_images(prompt_list[0])
    is_SD_preheated = True

sync_start_time = time.time()
images: list[Image.Image] = []
for prompt in tqdm(prompt_list, desc="Sync Generating"):
    for result in SD.generate_images(prompt):
        images.append(result.image)
sync_end_time = time.time()
print(
    f"Generated {len(images)} images with sync function in {sync_end_time - sync_start_time:.4f} seconds."
)

In [ ]:
async_start_time = time.time()
tasks = [SD.aio_generate_images(params, i) for i, params in enumerate(prompt_list)]
async_images: list[Image.Image] = (
    [Image.Image] * len(prompt_list) * prompt_list[0]["batch_size"]
)

for coro in async_tqdm(
    asyncio.as_completed(tasks), total=len(tasks), desc="Async Generating"
):
    result, index = await coro
    for i, img in enumerate(result):
        async_images[index * prompt_list[0]["batch_size"] + i] = img.image

async_end_time = time.time()
print(
    f"Generated {len(async_images)} images with async function in {async_end_time - async_start_time:.4f} seconds."
)

In [ ]:
if not os.path.exists("tmp"):
    os.makedirs("tmp")
for i, img in enumerate(async_images):
    with open(f"tmp/async_img{i}.webp", "wb") as f:
        img.save(f, format="WEBP")
# for i, img in enumerate(images):
#     with open(f"tmp/sync_img{i}.webp", "wb") as f:
#         img.save(f, format="WEBP")